In [1]:
import pandas as pd
from nsepy import get_history as gh
import datetime as dt
import yfinance as yf

In [2]:
import datetime as dt

start = dt.datetime(2021, 7, 1)
end = dt.datetime(2022, 1, 27)
symbol = 'HDFCBANK.NS'  # Stock symbol in uppercase

stk_data = yf.download(symbol, start=start, end=end)

[*********************100%%**********************]  1 of 1 completed


In [3]:
stk_data=stk_data[["Open","High","Low","Close"]]
dataset=stk_data

In [4]:
from sklearn.preprocessing import MinMaxScaler
Ms = MinMaxScaler()
data1= Ms.fit_transform(stk_data)
print("Len:",data1.shape)

Len: (143, 4)


In [5]:
data1=pd.DataFrame(data1,columns=["Open","High","Low","Close"])
data1

,Open,High,Low,Close
0,0.311864,0.244196,0.295880,0.255895
1,0.254237,0.200983,0.273408,0.232498
2,0.271017,0.252669,0.301686,0.287951
3,0.294915,0.372988,0.344569,0.432572
4,0.420339,0.391120,0.463296,0.450258
...,...,...,...,...
138,0.420339,0.372140,0.409550,0.372697
139,0.401525,0.334011,0.359925,0.337878
140,0.305085,0.338417,0.305618,0.384303
141,0.338813,0.309439,0.238015,0.255527


In [6]:

training_size = round(len(data1 ) * 0.80)
print(training_size)
X_train=data1[:training_size]
X_test=data1[training_size:]
print("X_train length:",X_train.shape)
print("X_test length:",X_test.shape)
y_train=data1[:training_size]
y_test=data1[training_size:]
print("y_train length:",y_train.shape)
print("y_test length:",y_test.shape)

114
X_train length: (114, 4)
X_test length: (29, 4)
y_train length: (114, 4)
y_test length: (29, 4)


In [7]:
import warnings
warnings.filterwarnings("ignore")

In [8]:
listt=["Close","High","Open","Low"]
#listt=["AQI_calculated","PM10","PM2.5","NOx","NO2","NO","NH3","SO2","CO",'year']

In [9]:
performance={"Model":[],"RMSE":[],"MaPe":[],"Lag":[],"Test":[]}


In [10]:
def combination(data1, listt):
    # Print the list of variables being used
    print("Variables used:", listt)
    
    # Extract the subset of the dataset with the selected variables
    datasetTwo = dataset[listt]
    
    # Define the number of observations for testing
    test_obs = 28
    
    # Split the data into training and testing sets
    train = datasetTwo[:-test_obs]
    test = datasetTwo[-test_obs:]
    
    # Import necessary libraries
    from statsmodels.tsa.statespace.varmax import VARMAX  
    from itertools import product
    
    # Initialize variables for storing best parameters
    best_aic = float('inf')
    best_params = None

    # Iterate over combinations of p and q values
    for p, q in product(range(1, 4), range(1, 4)):
        try:
            # Fit VARMAX model with current order
            model = VARMAX(train, order=(p, q))
            results = model.fit(maxiter=1000, disp=False)
            
            # Update best parameters if current AIC is lower
            aic = results.aic
            if aic < best_aic:
                best_aic = aic
                best_params = (p, q)
        except:
            continue

    # Print the best AIC and corresponding parameters
    print("Best AIC:", best_aic)
    print("Best Parameters (p, q):", best_params)

    # Fit VARMAX model with the best parameters
    model = VARMAX(train, order=best_params) 
    results = model.fit()
    
    # Forecast
    lagged_values = train.values[-max(best_params):]
    pred = results.forecast(y=lagged_values, steps=test_obs) 
    preds = pd.DataFrame(pred, columns=listt)
    preds.to_csv("varforecasted_{}.csv".format(test_obs))
    
    # Calculate performance metrics
    from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
    rmse = round(mean_squared_error(test, pred, squared=False))
    mape = mean_absolute_percentage_error(test, pred)
    
    # Store performance metrics in a dictionary
    performance = {
        "Model": [listt],
        "RMSE": [rmse],
        "MaPe": [mape],
        "Lag": [best_params],
        "Test": [test_obs]
    }
    
    # Create a DataFrame with performance metrics and return
    perf = pd.DataFrame(performance)
    return perf, results, pred


In [11]:
perf,result,pred= combination(data1, listt)
pred

Variables used: ['Close', 'High', 'Open', 'Low']


C:\Users\kathi\anaconda3\envs\tsa\Lib\site-packages\statsmodels\tsa\statespace\varmax.py:160: EstimationWarning: Estimation of VARMA(p,q) models is not generically robust, due especially to identification issues.
  warn('Estimation of VARMA(p,q) models is not generically robust,'
C:\Users\kathi\anaconda3\envs\tsa\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\kathi\anaconda3\envs\tsa\Lib\site-packages\statsmodels\tsa\statespace\varmax.py:160: EstimationWarning: Estimation of VARMA(p,q) models is not generically robust, due especially to identification issues.
  warn('Estimation of VARMA(p,q) models is not generically robust,'
C:\Users\kathi\anaconda3\envs\tsa\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency info

Best AIC: 3419.0249233909744
Best Parameters (p, q): (1, 1)


C:\Users\kathi\anaconda3\envs\tsa\Lib\site-packages\statsmodels\tsa\statespace\varmax.py:160: EstimationWarning: Estimation of VARMA(p,q) models is not generically robust, due especially to identification issues.
  warn('Estimation of VARMA(p,q) models is not generically robust,'
C:\Users\kathi\anaconda3\envs\tsa\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\kathi\anaconda3\envs\tsa\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\kathi\anaconda3\envs\tsa\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return

,Close,High,Open,Low
115,1505.081623,1512.863196,1495.985784,1488.179665
116,1507.175443,1519.608070,1505.031948,1494.540093
117,1508.552417,1520.504338,1506.583384,1495.685136
118,1509.836466,1521.987208,1507.924393,1497.024447
119,1511.060614,1523.229760,1509.211183,1498.228411
120,1512.223154,1524.451505,1510.430945,1499.389626
121,1513.328325,1525.602696,1511.591067,1500.489152
122,1514.378678,1526.699308,1512.693511,1501.535217
123,1515.377001,1527.740979,1513.741377,1502.529201
124,1516.325855,1528.731185,1514.737310,1503.473994


In [13]:
result

In [12]:
perf

,Model,RMSE,MaPe,Lag,Test
0,"[Close, High, Open, Low]",44,0.024663,"(1, 1)",28
